In [ ]:
import os
import pandas as pd
import numpy as np

# Import data

In [ ]:
myPath = "./presale/"

fileList = os.listdir(myPath)

# 合併所有預售屋檔案
dList = [pd.read_csv(myPath + f, skiprows=[1,2], error_bad_lines=False) for f in fileList]
preDF = pd.concat(dList)

In [ ]:
preDF.to_csv("./combine/presale.csv", index = False)

# Drop or Fill NA

In [ ]:
# 去除不用的欄位
noCol = ["土地移轉總面積平方公尺", "非都市土地使用分區", "非都市土地使用編定", "建築完成年月", "編號"]
newDF = preDF.drop(columns = noCol)

# "車位類別", "備註" => Fill NA
fillNA = {"車位類別": 0, "備註": 0}
df1 = newDF.fillna(value = fillNA)

In [ ]:
# 取得columns name
colList = list(df1.columns)

# 確認哪些欄位有空值
colnaList = []

for c in colList:
    if df1[c].isna().any():
        colnaList.append(c) 
        print(c)
        
for col in colnaList:
    print(df1[df1[col].isna()])

In [ ]:
# 若"鄉鎮市區", "土地區段位置建物區段門牌"兩欄位皆無資料，直接刪除  => Drop
df1_1 = df1.dropna(subset = ["鄉鎮市區", "土地區段位置建物區段門牌"], how = "all")

# 若"都市土地使用分區", "主要用途"兩欄位皆無資料，直接刪除  => Drop
df1_2 = df1_1.dropna(subset = ["都市土地使用分區", "主要用途"], how = "all")

# "移轉層次"空值，若遇"交易標的"是車位，補地下一層  => Fillna
df1_2.loc[df1_2.交易標的 == "車位", ["交易標的","移轉層次"]] = df1_2.loc[df1_2.交易標的 == "車位", ["交易標的", "移轉層次"]].fillna(value = {"移轉層次": "地下一層"})

# "移轉層次"空值，若遇"交易標的"為其他，則刪除  => Drop
df1_3 = df1_2.dropna(subset = ["移轉層次"], how = "any")

# 主要建材空值，取同"土地區段位置建物區段門牌" => Fillna
df1_3.loc[df1_3["主要建材"].isna(), ["土地區段位置建物區段門牌", "主要建材"]] = df1_3.loc[df1_3["主要建材"].isna(), ["土地區段位置建物區段門牌", "主要建材"]].fillna("鋼筋混凝土造")

# Fill"住" if "主要用途"="住家用"
df1_3.loc[((df1_3["都市土地使用分區"].isna()) & (df1_3["主要用途"] == "住家用")), ["都市土地使用分區", "土地區段位置建物區段門牌", "主要用途"]] = df1_3.loc[((df1_3["都市土地使用分區"].isna()) & (df1_3["主要用途"] == "住家用")), ["都市土地使用分區", "土地區段位置建物區段門牌", "主要用途"]].fillna(value = {"都市土地使用分區": "住"})

# Fill"其他" if "主要用途"="見其他登記事項"
df1_3.loc[((df1_3["都市土地使用分區"].isna()) & (df1_3["主要用途"] == "見其他登記事項")), ["都市土地使用分區", "土地區段位置建物區段門牌", "主要用途"]] = df1_3.loc[((df1_3["都市土地使用分區"].isna()) & (df1_3["主要用途"] == "見其他登記事項")), ["都市土地使用分區", "土地區段位置建物區段門牌", "主要用途"]].fillna(value = {"都市土地使用分區": "其他"})

# Fill "主要用途"="住家用"
df1_3.loc[df1_3["主要用途"].isna(), ["都市土地使用分區", "土地區段位置建物區段門牌", "主要用途"]] = df1_3.loc[df1_3["主要用途"].isna(), ["都市土地使用分區", "土地區段位置建物區段門牌", "主要用途"]].fillna(value = {"主要用途": "住家用"})
df1_3

# 欄位整理

In [ ]:
# check 欄位內容
def check_type(col):
    return list(df2[col].unique())


# 確認 "移轉層次"，要換成數字
fType = check_type("移轉層次")

# "移轉層次"換成數字表達
fList = [3,4,2,9,11,6,5,7,12,13,10,8,14,17,15,16,18,1,19,20,22,23,21,24,26,25,"見其他登記事項",27,28,29,-1,"all",36,34,30,31,32,"見使用執照",33,35]

df2 = df1_3.replace(fType,fList)


# 只需實際可用面積，所以要扣掉車位面積
df2["總坪數"] = (df2["建物移轉總面積平方公尺"] - df2["車位移轉總面積平方公尺"] ) * 0.3025

# 換算不含車位的每坪價格
df2["每坪單價"] = (df2["總價元"] - df2["車位總價元"]) / df2["總坪數"]

In [ ]:
# 篩選要使用的欄位

# 主要用途=住家用、住商用
# 建物型態=住宅大樓、華廈、套房
mainuse = ["住家用", "住商用"]
buildtype = ["住宅大樓(11層含以上有電梯)", "華廈(10層含以下有電梯)", "套房(1房1廳1衛)"]

df3_1 = df2[df2["主要用途"].isin(mainuse)]
df3_2 = df3_1[df3_1["建物型態"].isin(buildtype)]

In [ ]:
# 存出一版整理好 -- 所有新北市
df3_2.to_csv("./combine/presale.csv", index = False)

In [ ]:
# 選出要分析的四區 -- "新莊區", "新店區", "中和區", "板橋區"

district = ["新莊區", "新店區", "中和區", "板橋區"]

for i,d in enumerate(district):
     globals()["df4_" + str(i)] =  df3_2[df3_2["鄉鎮市區"] == d]
        
comDF = pd.concat([df4_0, df4_1, df4_2, df4_3], ignore_index = True)
comDF

In [ ]:
# 存出一版整理好--四區
comDF.to_csv("./combine/presale_4district.csv", index = False)